In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense, LSTM

In [25]:
# load the data into a pandas dataframe
df = pd.read_csv('USD_PLN Historical Data.csv', index_col='Date', parse_dates=True)
dfo = pd.read_csv('USD_PLN Historical Data.csv', index_col='Date', parse_dates=True)

# select the 'Price' column as the target variable
target_col = 'Price'

# scale the data
scaler = MinMaxScaler()
df[target_col] = scaler.fit_transform(df[target_col].values.reshape(-1,1))

# create the training and testing datasets
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# define the number of time steps and features
n_steps = 10
n_features = 1

# split the test data into validation set and set for predictions
validation_size = n_steps
if len(test_data) >= validation_size:
    validation_data = test_data.iloc[:validation_size]
    prediction_data = test_data.iloc[validation_size:]
else:
    validation_data = test_data
    prediction_data = None

def create_sequences(data, n_steps, n_features):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i])
        y.append(data[i])
    X = np.array(X)
    X = X.reshape((X.shape[0], n_steps, n_features))
    y = np.array(y)
    return X, y

# create sequences for training set
X_train, y_train = create_sequences(train_data[target_col].values.ravel(), n_steps, n_features)

# create sequences for validation set and set for predictions
X_val, y_val = create_sequences(validation_data[target_col].values, n_steps, n_features)
X_pred, y_pred = create_sequences(prediction_data[target_col].values, n_steps, n_features)



In [3]:
# define the model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(n_steps, n_features), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', run_eagerly=True) # add run_eagerly=True

# train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))




Epoch 1/20
28/28 [==============================] - 4s 131ms/step - loss: 0.0391
Epoch 2/20
28/28 [==============================] - 3s 123ms/step - loss: 0.0079
Epoch 3/20
28/28 [==============================] - 3s 120ms/step - loss: 0.0062
Epoch 4/20
28/28 [==============================] - 3s 122ms/step - loss: 0.0060
Epoch 5/20
28/28 [==============================] - 3s 119ms/step - loss: 0.0067
Epoch 6/20
28/28 [==============================] - 3s 122ms/step - loss: 0.0064
Epoch 7/20
28/28 [==============================] - 3s 121ms/step - loss: 0.0059
Epoch 8/20
28/28 [==============================] - 3s 120ms/step - loss: 0.0059
Epoch 9/20
28/28 [==============================] - 3s 121ms/step - loss: 0.0060
Epoch 10/20
28/28 [==============================] - 4s 121ms/step - loss: 0.0060
Epoch 11/20
28/28 [==============================] - 3s 120ms/step - loss: 0.0054
Epoch 12/20
28/28 [==============================] - 3s 121ms/step - loss: 0.0057
Epoch 13/20
28/28 [======

In [30]:
scaler = MinMaxScaler()
scaled_train_data = scaler.fit_transform(train_data)


ValueError: could not convert string to float: '20.54K'